In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xgboost as xgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from fastapi import FastAPI
import uvicorn

In [ ]:
sales_train = pd.read_csv("sales_train.csv")
items = pd.read_csv("items.csv")
shops = pd.read_csv("shops.csv")
item_categories = pd.read_csv("item_categories.csv")
test = pd.read_csv("test.csv")

print(sales_train.head())
print(items.head())
print(shops.head())
print(item_categories.head())
print(test.head())

         date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0  02.01.2013               0       59    22154      999.00           1.0
1  03.01.2013               0       25     2552      899.00           1.0
2  05.01.2013               0       25     2552      899.00          -1.0
3  06.01.2013               0       25     2554     1709.05           1.0
4  15.01.2013               0       25     2555     1099.00           1.0
                                           item_name  item_id  \
0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0   
1  !ABBYY FineReader 12 Professional Edition Full...        1   
2      ***В ЛУЧАХ СЛАВЫ   (UNV)                    D        2   
3    ***ГОЛУБАЯ ВОЛНА  (Univ)                      D        3   
4        ***КОРОБКА (СТЕКЛО)                       D        4   

   item_category_id  
0                40  
1                76  
2                40  
3                40  
4                40  
                        shop_name


- Recent Purchase Trends (Increase or decrease in sales over the last 3 months)
- Consistent, Minimum, and Maximum Selling Rates (Average, min, and max sales per shop-item pair)
- Rolling Statistics
- Returns Impact (How returns (item_cnt_day < 0) affect total sales)
- Seasonality (Sales patterns in previous Novembers or other months)

- Panelty when there is a return of any product.

In [ ]:
sales_train['date'] = pd.to_datetime(sales_train['date'], format='%d.%m.%Y')

penalty_multiplier = 1.1
sales_train['adjusted_item_cnt_day'] = sales_train['item_cnt_day'].apply(
    lambda x: x * penalty_multiplier if x < 0 else x
)
sales_monthly = sales_train.groupby(['date_block_num', 'shop_id', 'item_id'])['adjusted_item_cnt_day'].sum().reset_index()
sales_monthly.rename(columns={'adjusted_item_cnt_day': 'item_cnt_month'}, inplace=True)

print("Monthly Sales Data with Adjusted Returns:")
display(sales_monthly.head())

Monthly Sales Data with Adjusted Returns:


,date_block_num,shop_id,item_id,item_cnt_month
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0


In [6]:
display(sales_monthly)


,date_block_num,shop_id,item_id,item_cnt_month
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0
...,...,...,...,...
1609119,33,59,22087,6.0
1609120,33,59,22088,2.0
1609121,33,59,22091,1.0
1609122,33,59,22100,1.0


In [7]:
date_range = range(0, 34)
shop_ids = sales_train['shop_id'].unique()
item_ids = sales_train['item_id'].unique()

# Create the grid
grid = []
for date_block in date_range:
    for shop in shop_ids:
        for item in item_ids:
            grid.append([date_block, shop, item])
grid = pd.DataFrame(grid, columns=['date_block_num', 'shop_id', 'item_id'])

print("Grid of All Shop-Item Pairs:")
display(grid.head())
print("Grid Shape:", grid.shape)
print("Number of Unique Shops:", len(shop_ids))
print("Number of Unique Items:", len(item_ids))

Grid of All Shop-Item Pairs:


,date_block_num,shop_id,item_id
0,0,59,22154
1,0,59,2552
2,0,59,2554
3,0,59,2555
4,0,59,2564


Grid Shape: (44486280, 3)
Number of Unique Shops: 60
Number of Unique Items: 21807
